<a href="https://colab.research.google.com/github/GeorgeCrossIV/Product-Recommendation-Chatbot/blob/main/Vector_Search_Product_Recommendation_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with this notebook

- Create a new vector search enabled database in Astra. [astra.datastax.com](https://astra.datastax.com)
- For the easy path, name the keyspace in that database "vector_preview" (otherwise be prepared to modify the CQL in this notebook)
- Create a token with permissions to create tables
- Download your secure-connect-bundle zip file.
- Set up an open.ai API account and generate a key
- Update the Keys & Environment Variables cell in the notebook with information from the token you generated and the name of your secure connect bundle file.

# Setup

In [ ]:
!pip install openai pandas jupyter-datatables cassandra-driver

# Imports

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
from google.colab import userdata
import openai
import numpy
import pandas as pd

# Keys & Environment Variables

In [ ]:
# keys and tokens here
openai_api_key = userdata.get('openai_api_key')
openai.api_key = openai_api_key
cass_user = userdata.get('cass_user')
cass_pw = userdata.get('cass_pw')
scb_path = '/content/secure-connect-cassio-db.zip'
keyspace="chatbot"
table="products_table"
embed_data=False # set to True to reset the schema and create the embeddings

# Select a model to compute embeddings

In [ ]:
model_id = "text-embedding-ada-002"

# Connect to the Cluster

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace('vector_preview')
session

# Drop / Create Schema

In [ ]:
# only use this to reset the schema
if embed_data:
  session.execute(f"""DROP INDEX IF EXISTS {keyspace}.openai_desc""")
  session.execute(f"""DROP TABLE IF EXISTS {keyspace}.{table}""")

In [ ]:
# # Create Table
if embed_data:
  session.execute(f"""
  CREATE TABLE IF NOT EXISTS {keyspace}.{table}
  (product_id int,
  chunk_id int,

  product_name text,
  description text,
  price text,

  openai_description_embedding vector<float, 1536>,
  minilm_description_embedding vector<float, 384>,

  PRIMARY KEY (product_id, chunk_id))
  """)

  # # Create Index
  session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS openai_desc ON {keyspace}.{table} (openai_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

# Load the table with data and create text embeddings

In [ ]:
if embed_data:
  !wget https://raw.githubusercontent.com/GeorgeCrossIV/Product-Recommendation-Chatbot/main/ProductDataset.csv
  products_list = pd.read_csv('ProductDataset.csv')
  products_list

This next step creates embeddings and inserts the data. It will take a few minutes to complete

In [ ]:
if embed_data:
  for id, row in products_list.iterrows():
    # Create Embedding for each conversation row, save them to the database
    text_chunk_length = 2500
    text_chunks = [row.description[i:i + text_chunk_length] for i in range(0, len(row.description), text_chunk_length)]
    for chunk_id, chunk in enumerate(text_chunks):
      pricevalue = row.price if isinstance(row.price, str) else ""
      full_chunk = f"{chunk} price: {pricevalue}"
      embedding = openai.embeddings.create(input=full_chunk, model=model_id).data[0].embedding
      query = SimpleStatement(
                  f"""
                  INSERT INTO {keyspace}.{table}
                  (product_id, chunk_id, product_name, description, price, openai_description_embedding)
                  VALUES (%s, %s, %s, %s, %s, %s)
                  """
              )
      display(row)

      session.execute(query, (row.product_id, chunk_id, row.product_name, row.description, pricevalue, embedding))



---


# Start using the index

In the steps up to this point, we have been creating a schema and loading the table with data, including embeddings we generated through the OpenAI Embedding API.
Now we are going to query that table and use the results to give ChatGPT some context to support it's response.

# Convert a query string into a text embedding to use as part of the query

This is where the real fun starts.  Provide a question or request to be used as the query.  The source sample database is mostly consumer electronics and appliances, so imagine you're talking to a customer service rep at Best Buy or another electronics store.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [ ]:
customer_input = "What equipement would you recommend for a computer workstation setup costing less than $2000?"

embedding = openai.embeddings.create(input=customer_input, model=model_id).data[0].embedding
display(embedding)

Let's take a look at what a query against a vector index could look like.  The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

In [ ]:
query = SimpleStatement(
    f"""
    SELECT *
    FROM {keyspace}.{table}
    ORDER BY openai_description_embedding ANN OF {embedding} LIMIT 5;
    """
    )
display(query)

# Find the top 5 results using ANN Similarity

In [ ]:
results = session.execute(query)
top_5_products = results._current_rows

for row in top_5_products:
  print(f"""{row.product_id}, {row.product_name}, {row.description}, {row.price}\n""")

# Ask ChatGPT for some help (Prompt Engineering)

- Here we build a prompt with which we'll query ChatGPT.  Note the "roles" in this little conversation give the LLM more context about who that part of the conversation is coming from.
- This may take 10-20 seconds to return, so be patient.

In [ ]:

message_objects = []
message_objects.append({"role":"system",
                        "content":"You're a chatbot helping customers with questions and helping them with product recommendations"})

message_objects.append({"role":"user",
                        "content": customer_input})

message_objects.append({"role":"user",
                        "content": "Please give me a detailed explanation of your recommendations"})

message_objects.append({"role":"user",
                        "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})

message_objects.append({"role": "assistant",
                        "content": "I found these 3 products I would recommend"})

products_list = []

for row in top_5_products:
    brand_dict = {'role': "assistant", "content": f"{row.description}"}
    products_list.append(brand_dict)

message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})

completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)
print(completion.choices[0].message.content)
